 Детальная и комплексная кластеризация спиртных напитков объемом 0.7 л, которые продаются в сети ["КуулКлевер"](https://www.coolclever.ru/catalog/otdokhni/all?filters=11099%255B0%255D%3D10%2611099%255B1%255D%3D11%2611099%255B2%255D%3D9%2611099%255B3%255D%3D12%2611099%255B4%255D%3D17%2611099%255B5%255D%3D13%2611099%255B6%255D%3D14%2611099%255B7%255D%3D15%2611099%255B8%255D%3D16%2611099%255B9%255D%3D43%2611109%255B0%255D%3D0.700&sort=cheap&page=9), с помощью различных методов машинного обучения.


In [133]:
# Объявляю нужные библиотеки
import pandas as pd # Библиотека для работы с фреймворками
from sklearn.preprocessing import LabelEncoder # Объект разметки текстовых данных

In [134]:
# Загрузка датасета из csv-файла
df = pd.read_csv('CoolClever.csv', sep=',')
df

,Напиток,Цена,Страна,Крепость,Выдержка,Подарочная упаковка
0,Водка,490,Россия,40.0,0,0
1,Водка,490,Россия,40.0,0,0
2,Водка,490,Россия,40.0,0,0
3,Водка,539,Россия,40.0,0,0
4,Водка,539,Россия,40.0,0,0
...,...,...,...,...,...,...
88,Джин,2800,Северная Ирландия,42.5,0,0
89,Виски,3590,Шотландия,40.0,12,0
90,Виски,3650,Ирландия,43.0,3,0
91,Коньяк,2549,Франция,40.0,3,1


In [135]:
# Разметка текстовых данных в таблице
le = LabelEncoder()
df['Напиток'] = le.fit_transform(df['Напиток'])
df['Страна'] = le.fit_transform(df['Страна'])
df

,Напиток,Цена,Страна,Крепость,Выдержка,Подарочная упаковка
0,4,490,7,40.0,0,0
1,4,490,7,40.0,0,0
2,4,490,7,40.0,0,0
3,4,539,7,40.0,0,0
4,4,539,7,40.0,0,0
...,...,...,...,...,...,...
88,5,2800,9,42.5,0,0
89,3,3590,13,40.0,12,0
90,3,3650,3,43.0,3,0
91,7,2549,10,40.0,3,1


In [136]:
# Информация о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Напиток              93 non-null     int64  
 1   Цена                 93 non-null     int64  
 2   Страна               93 non-null     int64  
 3   Крепость             93 non-null     float64
 4   Выдержка             93 non-null     int64  
 5   Подарочная упаковка  93 non-null     int64  
dtypes: float64(1), int64(5)
memory usage: 4.5 KB


In [137]:
# Описание датасета
df.describe()

,Напиток,Цена,Страна,Крепость,Выдержка,Подарочная упаковка
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000
mean,4.516129,1825.032258,7.623656,39.540860,2.182796,0.129032
std,2.560625,860.082193,4.141293,3.795284,2.714479,0.337053
min,0.000000,490.000000,0.000000,17.000000,0.000000,0.000000
25%,3.000000,1199.000000,5.000000,40.000000,0.000000,0.000000
50%,4.000000,1766.000000,8.000000,40.000000,2.000000,0.000000
75%,6.000000,2490.000000,10.000000,40.000000,3.000000,0.000000
max,10.000000,3650.000000,13.000000,47.000000,12.000000,1.000000


In [138]:
# Корреляционные зависимости
corr = df.corr()
corr

,Напиток,Цена,Страна,Крепость,Выдержка,Подарочная упаковка
Напиток,1.000000,-0.102334,-0.400715,-0.278007,-0.457839,-0.040220
Цена,-0.102334,1.000000,0.195374,0.099845,0.520412,0.473474
Страна,-0.400715,0.195374,1.000000,0.120007,0.581503,0.260995
Крепость,-0.278007,0.099845,0.120007,1.000000,0.142651,0.097799
Выдержка,-0.457839,0.520412,0.581503,0.142651,1.000000,0.318469
Подарочная упаковка,-0.040220,0.473474,0.260995,0.097799,0.318469,1.000000
